<img src="http://openenergy-platform.org/static/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="100" width="100"  align="left"/>

# OpenEnergyPlatform
<br><br>

## Collect data from OEP using oedialect
Repository: https://github.com/openego/oedialect

Please report bugs and improvements here: https://github.com/OpenEnergyPlatform/oedialect/issues <br>
Using SAIO functions to setup SqlAlchemy classes: https://github.com/coroa/saio

In [ ]:
__copyright__ = "Reiner Lemoine Institut"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "Ludee"

In [ ]:
import oedialect
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker

import pandas as pd
import numpy as np
import getpass
import matplotlib.pyplot as plt
import saio

import pyam

%matplotlib inline

## Connection to OEP

If we want to upload data to the OEP we first need to connect to it, using our OEP user name and token. 
Note: You ca view your token on your OEP profile page after logging in.  


In [ ]:
# White spaces in the username are fine!
user = input('Enter OEP-username:')
token = getpass.getpass('Token:')

Now we'll create an sql-alchemy-engine. The engine is what 'speaks' oedialect to the data base api. We need to tell it where the data base is and pass our credentials.


In [ ]:
# Create Engine:
oep_url = 'openenergy-platform.org' #'193.175.187.164' #'oep.iks.cs.ovgu.de'
oed = f'postgresql+oedialect://{user}:{token}@{oep_url}'

engine = sa.create_engine(oed)
metadata = sa.MetaData(bind=engine)
print(metadata)

## Setup ORM

In [ ]:
saio.register_schema("boundaries", engine)
saio.register_schema("grid", engine)
saio.register_schema("supply", engine)
saio.register_schema("model_draft", engine)


In [ ]:
saio.register_schema("scenario", engine)

In [ ]:
# Included in SAIO
# from sqlalchemy.ext.declarative import declarative_base
# Base = declarative_base()
# metadata = Base.metadata
# metadata.bind=engine

## Get data

### Bundesländer

In [ ]:
from saio.boundaries import bkg_vg250_2_lan as BkgVg250Lan

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    df_lan = saio.as_pandas(session.query(BkgVg250Lan))
    session.commit()
    print('Querry successful!')
except Exception as e:
    session.rollback()
    raise
    print('Querry incomplete!')
finally:
    session.close()

In [ ]:
ax = df_lan.plot(figsize=(5, 5), color='white', edgecolor='black')

# Download Scenario data

In [ ]:
from saio.scenario import ksz2050_r2_ks95_primary_and_final_energy_consumption as Ksz2050pfec

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    df_lan = saio.as_pandas(session.query(Ksz2050pfec))
    session.commit()
    print('Querry successful!')
except Exception as e:
    session.rollback()
    raise
    print('Querry incomplete!')
finally:
    session.close()

In [ ]:
df_lan.head()

### Gemeinden

In [ ]:
from saio.boundaries import bkg_vg250_6_gem as BkgVg250Gem
from saio.boundaries import bkg_vg250_4_krs as BkgVg250Krs

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    df_krs = saio.as_pandas(session.query(BkgVg250Krs))
    session.commit()
    print('Querry successful!')
except Exception as e:
    session.rollback()
    raise
    print('Querry incomplete!')
finally:
    session.close()

In [ ]:
ax = df_lan.plot(figsize=(5, 5), color='white', edgecolor='black')
df_krs.plot(ax=ax, alpha=0.5, color='white', edgecolor='grey')

### eGoDP - EHV

In [ ]:
from saio.grid import ego_dp_ehv_substation as eGoEHVSub
from saio.grid import ego_dp_ehv_griddistrict as eGoEHVGD

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    df_ehvsub = saio.as_pandas(session.query(eGoEHVSub), geometry='point')
    df_ehvgd = saio.as_pandas(session.query(eGoEHVGD))
    session.commit()
    print('Querry successful!')
except Exception as e:
    session.rollback()
    raise
    print('Querry incomplete!')
finally:
    session.close()

In [ ]:
fig, ax = plt.subplots(1, figsize=(20, 20))
ax.set_aspect('equal')
ax.set_axis_off()
ax.set_title('open_eGo - EHV')

df_ehvgd.to_crs({'init': 'epsg:3035'}).plot(ax=ax, color='white', edgecolor='grey', linewidth=1)

df_lan.to_crs({'init': 'epsg:3035'}).plot(ax=ax, color='white', alpha=.5, edgecolor='black', linewidth=2)

df_ehvsub.to_crs({'init': 'epsg:3035'}).plot(ax=ax, marker='o', color='orange', markersize=5)

plt.axis('equal')

### eGoDP - MV

In [ ]:
from saio.grid import ego_dp_hvmv_substation as eGoHVMVSub
from saio.grid import ego_dp_mv_griddistrict as eGoMVGD

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    df_hvmvsub = saio.as_pandas(session.query(eGoHVMVSub))
    df_mvgd = saio.as_pandas(session.query(eGoMVGD))
    session.commit()
    print('Querry successful!')
except Exception as e:
    session.rollback()
    raise
    print('Querry incomplete!')
finally:
    session.close()

In [ ]:
fig, ax = plt.subplots(1, figsize=(20, 20))
ax.set_aspect('equal')
ax.set_axis_off()
ax.set_title('open_eGo - MV')

df_mvgd.to_crs({'init': 'epsg:3035'}).plot(ax=ax, color='white', edgecolor='grey', linewidth=1)

df_lan.to_crs({'init': 'epsg:3035'}).plot(ax=ax, color='white', alpha=.5, edgecolor='black', linewidth=2)

df_hvmvsub.to_crs({'init': 'epsg:3035'}).plot(ax=ax, marker='o', color='orange', markersize=5)

plt.axis('equal')

### eGoDP - Supply 

In [ ]:
from saio.supply import ego_dp_conv_powerplant as eGoPPConv
from saio.supply import ego_dp_res_powerplant as eGoPPRes

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
session.query(eGoPPConv).filter(eGoPPConv.scenario == 'Status Quo').all()

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    sq = session.query(eGoPPConv).filter(eGoPPConv.scenario == 'Status Quo')
    df_ppconv = saio.as_pandas(sq)
    
    sq = session.query(eGoPPRes).filter(eGoPPRes.scenario == 'Status Quo')
    df_ppres = saio.as_pandas(sq)
    session.commit()
    print('Querry successful!')
except Exception as e:
    session.rollback()
    raise
    print('Querry incomplete!')
finally:
    session.close()

### MVIEW - ERROR!

ToDo: Include MViews autoload in SAIO

In [ ]:
from saio.supply import ego_dp_conv_powerplant_sq_mview as eGoPPConvSQ
from saio.supply import ego_dp_res_powerplant_sq_mview as eGoPPResSQ